In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import time

fico_input = pd.read_csv("UCSD_FICO_09/DataminingContest2009.Task1.Train.Inputs")

fico_input.head()

,amount,hour1,state1,zip1,field1,domain1,field2,hour2,flag1,total,field3,field4,field5,indicator1,indicator2,flag2,flag3,flag4,flag5
0,12.95,0,CA,925,3,AOL.COM,1,0,1,12.95,-4276,7,0,1,0,1,1,0,1
1,11.01,0,CA,925,3,AOL.COM,1,0,1,11.01,-4276,7,0,1,0,1,1,0,1
2,38.85,0,CA,928,3,HOTMAIL.COM,1,0,0,38.85,2602,21,1,0,0,0,0,0,1
3,25.90,0,NJ,77,0,AOL.COM,1,0,0,25.90,4139,6,0,0,0,1,1,0,1
4,12.95,0,CA,945,3,YAHOO.COM,0,0,1,12.95,3826,9,1,0,0,1,0,0,1


In [2]:
# fico_input = pd.get_dummies(fico_input, drop_first=True)

fico_input.apply(pd.Series.nunique)

amount           88
hour1            24
state1           53
zip1            899
field1            5
domain1        9809
field2            2
hour2            24
flag1             2
total            88
field3        15786
field4           38
field5           26
indicator1        2
indicator2        2
flag2             2
flag3             2
flag4             2
flag5            36
dtype: int64

In [3]:
fico_input = fico_input.drop(['state1', 'total'], axis = 1)
fico_input = pd.get_dummies(fico_input, drop_first = True)
# fico_input.head()

In [4]:
print(fico_input.shape)

(94682, 9824)


In [5]:
fico_target = pd.read_csv("UCSD_FICO_09/DataminingContest2009.Task1.Train.Targets",header=None)

fico_target.head()

,0
0,0
1,0
2,0
3,0
4,0


In [6]:
prop_default = np.sum(fico_target.values) / len(fico_target) * 100

print("Percent of default records in training set: {0:.2f}%".format(prop_default))

Percent of default records in training set: 2.21%


In [7]:
X_train, X_test, y_train, y_test = train_test_split(fico_input, fico_target.iloc[:,0], random_state=0)

In [8]:
# approx_weights = np.round(len(X_train) / (2 * np.bincount(y_train)))

# weights1 = np.arange(approx_weights[0]-0.5, approx_weights[0]+0.5, .1)
# weights2 = np.arange(approx_weights[1]-0.5, approx_weights[1]+0.5, .1)

# weights1_all, weights2_all = np.meshgrid(weights1, weights2)

# weights1_all = weights1_all.ravel()
# weights2_all = weights2_all.ravel()

# weight_dict_list = []
# for i in range(len(weights1_all)):
#     weight_dict_list.append({0: weights1_all[i], 1: weights2_all[i]})

# param_grid = {'n_estimators': [int(a) for a in np.linspace(50, 100, 2)],
#               'max_depth': np.linspace(1,3,3),
#                 'class_weight': weight_dict_list}

param_grid = {'n_estimators': [int(a) for a in np.linspace(50, 100, 2)],
              'max_depth': np.linspace(1,3,3),
                'class_weight': ['balanced']}

clf_cv = GridSearchCV(ensemble.RandomForestClassifier(), param_grid)

print("Running....")
tic = time.time()
clf_cv.fit(X_train, y_train)

toc = time.time()
print("Training time: {0}s".format(toc - tic))

Running....
Training time: 684.1935658454895s


In [9]:
clf = clf_cv.best_estimator_
print(clf)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

RandomForestClassifier(bootstrap=True, class_weight={0: 1.0, 1: 22.5},
            criterion='gini', max_depth=1.0, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
             precision    recall  f1-score   support

          0       0.98      1.00      0.99     23141
          1       0.00      0.00      0.00       530

avg / total       0.96      0.98      0.97     23671

[[23141     0]
 [  530     0]]


/Users/steve/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
weight_dict_list

[{0: 0.5, 1: 22.5}, {0: 1.0, 1: 22.5}, {0: 0.5, 1: 23.0}, {0: 1.0, 1: 23.0}]